In [1]:
import numpy as np 
import os 
import json 
import cv2
from glob import glob
from pprint import pprint
import numpy as np
import scipy.linalg
import cv2
import numpy as np
import os

In [2]:
box_path_list = sorted(list(glob("/aidata/anders/objects/landmarks/ttlnmks/bboxes_*.npy")))
lnmk_path_list = sorted(list(glob("/aidata/anders/objects/landmarks/ttlnmks/lnmks_*.npy")))

In [3]:
# vim: expandtab:ts=4:sw=4
chi2inv95 = {
    1: 3.8415,
    2: 5.9915,
    3: 7.8147,
    4: 9.4877,
    5: 11.070,
    6: 12.592,
    7: 14.067,
    8: 15.507,
    9: 16.919
}


class KalmanFilter(object):
    def __init__(self):
        self.lnmk_dims = 10
        ndim, dt = self.lnmk_dims, 1.
        # Create Kalman filter model matrices.
        self._motion_mat = np.eye(2 * ndim, 2 * ndim)
        for i in range(ndim):
            self._motion_mat[i, ndim + i] = dt
        self._update_mat = np.eye(ndim, 2 * ndim)
        self._std_weight_position = 1. / 20
        self._std_weight_velocity = 1. / 160

    def initiate(self, measurement, obj_h=735):

        mean_pos = measurement
        mean_vel = np.zeros_like(mean_pos)
        mean = np.r_[mean_pos, mean_vel]
        
        std = list(2 * self._std_weight_position * measurement) + list(
            10 * self._std_weight_velocity * measurement)
        
        covariance = np.diag(np.square(std))
        return mean, covariance

    def predict(self, mean, covariance, obj_h):
        std_pos = self._std_weight_position * mean[:10]
        std_vel = self._std_weight_velocity * mean[:10]
        
        motion_cov = np.diag(np.square(np.r_[std_pos, std_vel]))

        mean = np.dot(self._motion_mat, mean)
        covariance = np.linalg.multi_dot(
            (self._motion_mat, covariance, self._motion_mat.T)) + motion_cov
        
        return mean, covariance

    def project(self, mean, covariance):
        std = self._std_weight_position * mean[:10]
        innovation_cov = np.diag(np.square(std))
        mean = np.dot(self._update_mat, mean)
        covariance = np.linalg.multi_dot(
            (self._update_mat, covariance, self._update_mat.T))
        return mean, covariance + innovation_cov

    def update(self, mean, covariance, measurement):
        projected_mean, projected_cov = self.project(mean, covariance)
        
        chol_factor, lower = scipy.linalg.cho_factor(projected_cov,
                                                     lower=True,
                                                     check_finite=False)
        kalman_gain = scipy.linalg.cho_solve((chol_factor, lower),
                                             np.dot(covariance,
                                                    self._update_mat.T).T,
                                             check_finite=False).T
        innovation = measurement - projected_mean
        new_mean = mean + np.dot(innovation, kalman_gain.T)
        new_covariance = covariance - np.linalg.multi_dot(
            (kalman_gain, projected_cov, kalman_gain.T))
        return new_mean, new_covariance

In [13]:
class PoseEstimator:
    """Estimate head pose according to the facial landmarks"""
    def __init__(self, img_size=(480, 640)):
        self.size = img_size
        # Camera internals
        self.focal_length = self.size[1]
        self.camera_center = (self.size[1] / 2, self.size[0] / 2)
        self.camera_matrix = np.array(
            [[self.focal_length, 0, self.camera_center[0]],
             [0, self.focal_length, self.camera_center[1]], [0, 0, 1]],
            dtype="double")
        # Assuming no lens distortion
        self.dist_coeefs = np.zeros((4, 1))
        # Rotation vector and translation vector
        self.r_vec = np.array([[0.01891013], [0.08560084], [-3.14392813]])

        self.t_vec = np.array([[-14.97821226], [-10.62040383],
                               [-2053.03596872]])

        """
         if os.path.isfile('/home/anders/projects/lite_model/behaviornet/assets/calibration.yml'):
            mtx, dist, rvecs, tvecs = self.load_coefficients(
                '/home/anders/projects/lite_model/behaviornet/assets/calibration.yml')
            self.camera_matrix = mtx
            self.dist_coeefs = dist
        """
        #     self.r_vec = rvecs
        #     self.t_vec = tvecs
        # 3D model points.
        self.model_points_68 = self._get_full_model_points()

    def load_coefficients(self, path):
        '''Loads camera matrix and distortion coefficients.'''
        # FILE_STORAGE_READ
        cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_READ)
        # note we also have to specify the type to retrieve other wise we only get a
        # FileNode object back instead of a matrix
        camera_matrix = cv_file.getNode('K').mat()
        dist_matrix = cv_file.getNode('D').mat()
        rvecs = cv_file.getNode('R').mat()
        tvecs = cv_file.getNode('T').mat()
        cv_file.release()
        return [camera_matrix, dist_matrix, rvecs, tvecs]

    def _get_full_model_points(self, filename='/home/anders/projects/lite_model/behaviornet/assets/model.txt'):
        """Get all 68 3D model points from file"""
        raw_value = []
        with open(filename) as file:
            for line in file:
                raw_value.append(line)
        model_points = np.array(raw_value, dtype=np.float32)
        model_points = np.reshape(model_points, (3, -1)).T
        # Transform the model into a front view.
        model_points[:, 2] *= -1
        # fetch lnmk25
        lnmks_68 = [
            0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
            19, 20, 21, 22, 23, 24, 25, 26, 36, 37, 38, 39, 40, 41, 42, 43, 44,
            45, 46, 47, 27, 28, 29, 30, 31, 32, 33, 34, 35, 48, 49, 50, 51, 52,
            53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67
        ]
        # correct to our facial landmarks
        model_points = model_points[lnmks_68]
        lnmk_scheme = [
            27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 42, 48, 50, 51,
            52, 54, 56, 57, 58
        ]
        model_points = model_points[lnmk_scheme]
        LE = np.mean(model_points[:6], axis=0, keepdims=True)
        RE = np.mean(model_points[6:12], axis=0, keepdims=True)
        N = model_points[13:14]  #13
        LM = model_points[14:15]
        RM = model_points[18:19]
        model_points = np.concatenate([LE, RE, N, LM, RM], axis=0)

        #----------------------------

        model_points = np.asarray(model_points)
        return model_points

    def solve_pose_by_68_points(self, image_points):
        """
        Solve pose from all the 68 image points
        Return (rotation_vector, translation_vector) as pose.
        """
        if self.r_vec is None:
            (_, rotation_vector, translation_vector) = cv2.solvePnP(
                self.model_points_68, image_points, self.camera_matrix,
                self.dist_coeefs)
            self.r_vec = rotation_vector
            self.t_vec = translation_vector
        print(image_points)
        (_, rotation_vector,
         translation_vector) = cv2.solvePnP(self.model_points_68,
                                            image_points,
                                            self.camera_matrix,
                                            self.dist_coeefs,
                                            rvec=self.r_vec,
                                            tvec=self.t_vec,
                                            useExtrinsicGuess=True,
                                            flags = cv2.SOLVEPNP_ITERATIVE)
        return (rotation_vector, translation_vector)
width, height = 1280, 720
pose_estimator = PoseEstimator(img_size=(height, width))

In [14]:




kf_lnmk = KalmanFilter()
temp_lnmk = None
mean_lnmk = None
for i, (box_path, lnmk_path) in enumerate(zip(box_path_list, lnmk_path_list)):
    if (i==2):

        bboxes= np.load(box_path)
        bboxes = bboxes.reshape([15, 6])
        valid_mask = np.all(np.isfinite(bboxes), axis=-1)
        bboxes = bboxes[valid_mask]
        lnmks= np.load(lnmk_path)
        lnmks = lnmks.reshape([-1, 11])
        hws = bboxes[:, 2:4] - bboxes[:, :2]
        areas = hws[:, 0] * hws[:, 1]
        if len(areas) != 0:
            idx = np.argmax(areas, axis=0)
            bbox = bboxes[idx]
            tl, br = bbox[:2], bbox[2:4]
            y1, x1 = tl
            y2, x2 = br
            lnmk_scs = lnmks[:, -1]
            max_idx = np.argmax(lnmk_scs)
            lnmk = lnmks[max_idx, :-1].reshape([5, 2])
            logical_y = np.logical_and(y1 < lnmk[:, :1],
                                       lnmk[:, :1] < y2)
            logical_x = np.logical_and(x1 < lnmk[:, 1:],
                                       lnmk[:, 1:] < x2)
            logical_yx = np.concatenate([logical_y, logical_x], axis=-1)
            inside_masks = np.all(logical_yx, axis=-1)
            lnmk = lnmk[inside_masks]
            temp_lnmk = lnmk.reshape([-1])
            if mean_lnmk is None:
                mean_lnmk, covariance_lnmk = kf_lnmk.initiate(temp_lnmk, (y2 - y1))
                lnmks = mean_lnmk[:10]
            elif(i ==1):
                mean_lnmk, covariance_lnmk = kf_lnmk.predict(
                                        mean_lnmk, covariance_lnmk, (y2 - y1))

                mean_lnmk, covariance_lnmk = kf_lnmk.update(
                                        mean_lnmk, covariance_lnmk, temp_lnmk)
                lnmks = mean_lnmk[:10]
            elif(i ==2):

                mean_lnmk, covariance_lnmk = kf_lnmk.predict(
                                        mean_lnmk, covariance_lnmk, (y2 - y1))

                mean_lnmk, covariance_lnmk = kf_lnmk.update(
                                        mean_lnmk, covariance_lnmk, temp_lnmk)

                lnmks = mean_lnmk[:10]
            lnmks = lnmks.reshape([5, 2])
            marks=  lnmks[..., ::-1]
            pose = pose_estimator.solve_pose_by_68_points(marks)

            r_mat, _ = cv2.Rodrigues(pose[0])
            p_mat = np.hstack((r_mat, np.array([[0], [0], [0]])))
            _, _, _, _, _, _, u_angle = cv2.decomposeProjectionMatrix(p_mat)
            pitch, yaw, roll = u_angle.flatten()
            if roll > 0:
                    roll = 180 - roll
            elif roll < 0:
                roll = -(180 + roll)
            print(pitch, yaw, roll)
            xxx
            
            


[[724.8267  307.9425 ]
 [873.3116  322.55658]
 [808.      382.5    ]
 [738.9631  453.55182]
 [845.8246  461.49222]]
-1.840414782531409 -0.1472769949432815 -4.0862451110157565


NameError: name 'xxx' is not defined